# MNIST Dataset Processing and Neural Network

In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler


2024-12-28 00:11:12.907197: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-28 00:11:12.909323: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-28 00:11:12.918471: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-28 00:11:12.944137: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735344672.980653    1820 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735344672.99

## Step 1: Download and Process MNIST Dataset

In [2]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Resize images to 16x16 (256 pixels)
x_train = x_train[:, ::2, ::2]  # Downsample by skipping every other pixel
x_test = x_test[:, ::2, ::2]

# Normalize pixel values to range [0.0, 1.0] (0.0 for white, 1.0 for black)
x_train = 1 - x_train / 255.0
x_test = 1 - x_test / 255.0

# Flatten images into 256-pixel vectors
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step 


## Step 2: Save Processed Data to CSV Files

In [3]:
# Save input data to input.csv
input_df = pd.DataFrame(x_train_flat)
input_df.to_csv("input.csv", index=False, header=False)


# Save target data to target.csv
target_df = pd.DataFrame(y_train_onehot)
target_df.to_csv("target.csv", index=False, header=False)

print("Saved input.csv and target.csv.")

Saved input.csv and target.csv.


## Step 2.1 Install Rust machine learning library

In [2]:
# clone github.com/mpohl100/ml_rust and cargo build --release

import os
import subprocess

def clone_and_build(repo_url, branch="main"):
    """
    Clones a GitHub repository and runs `cargo build --release`.

    Args:
        repo_url (str): The GitHub repository URL to clone.
        branch (str): The branch to checkout. Default is "main".
    """
    try:
        # Extract repository name from URL
        repo_name = repo_url.split("/")[-1].replace(".git", "")
        
        # Check if the repository already exists
        if not os.path.exists(repo_name):
            print(f"Cloning {repo_url} into {os.getcwd()}...")
            subprocess.run(
                ["git", "clone", "--branch", branch, repo_url],
                check=True
            )
        else:
            print(f"Repository {repo_name} already exists. Skipping clone.")
            # do a git pull
            os.chdir(repo_name)
            print("Running `git pull`...")
            subprocess.run(["git", "pull"], check=True)
            os.chdir("..")

        # Change into the repository directory
        repo_path = os.path.join(os.getcwd(), repo_name)
        os.chdir(repo_path)

        # Run `cargo build --release`
        print("Running `cargo build --release`...")
        subprocess.run(["cargo", "build", "--release"], check=True)

        print("Build completed successfully!")

    except subprocess.CalledProcessError as e:
        print(f"An error occurred while executing a command: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        # Change back to the original directory
        os.chdir("..")

repo_url = "https://github.com/mpohl100/ml_rust.git"
clone_and_build(repo_url)


Repository ml_rust already exists. Skipping clone.
Running `git pull`...


hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.
From https://github.com/mpohl100/ml_rust
   7049782..66a841e  main       -> origin/main


Updating 7049782..66a841e
Fast-forward
 Cargo.toml                       |   4 ++
 src/evaluate.rs                  |  16 +++---
 src/neural/activation/softmax.rs |  38 ++++----------
 src/neural/nn/neuralnet.rs       |   4 +-
 src/predict.rs                   | 109 +++++++++++++++++++++++++++++++++++++++
 5 files changed, 133 insertions(+), 38 deletions(-)
 create mode 100644 src/predict.rs
Running `cargo build --release`...


   Compiling learn v0.1.0 (/workspaces/ml-jupyter-notebook/ml_rust)


Build completed successfully!


    Finished `release` profile [optimized] target(s) in 3.62s


## Step 2.2 Train neural network based on shape file

In [ ]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
shape_file = "nn_shape.yaml"
training_verification_ratio = 0.7
learning_rate = 0.05
epochs = 10
tolerance = 0.2
batch_size = 32

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/train",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--shape-file", shape_file,
    "--training-verification-ratio", str(training_verification_ratio),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--batch-size", str(batch_size),
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


Inputs: 59999 x 784
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 784, output_size: 128 }, activation: ActivationData { activation_type: ReLU, temperature: None } }, LayerShape { layer_type: Dense { input_size: 128, output_size: 64 }, activation: ActivationData { activation_type: ReLU, temperature: None } }, LayerShape { layer_type: Dense { input_size: 64, output_size: 10 }, activation: ActivationData { activation_type: Softmax, temperature: Some(1.0) } }] }
Epoch: 0
Epoch 0: Loss 1.7892917756940343, Accuracy 81.88171146928879%
Epoch: 1
Epoch 1: Loss 1.8198528536393723, Accuracy 81.80147146357406%
Epoch: 2
Epoch 2: Loss 1.8198528536393723, Accuracy 81.80147146357406%
Epoch: 3
Epoch 3: Loss 1.8198528536393723, Accuracy 81.80147146357406%
Epoch: 4
Epoch 4: Loss 1.8198528536393723, Accuracy 81.80147146357406%
Epoch: 5


KeyboardInterrupt: 

## Step 3: Create a Neural Network using Rust neural network generator

In [3]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
training_verification_ratio = 0.7
learning_rate = 0.1
epochs = 100
tolerance = 0.1
batch_size = 32
num_generations = 100
log_level = 1
population_size = 4
num_offsprings = 10

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/nn_generator",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--training-verification-ratio", str(training_verification_ratio),
    "--learning-rate", str(learning_rate),
    "--epochs", str(epochs),
    "--tolerance", str(tolerance),
    "--batch-size", str(batch_size),
    "--num-generations", str(num_generations),
    "--log-level", str(log_level),
    "--population-size", str(population_size),
    "--num-offsprings", str(num_offsprings)
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model training completed successfully and saved in:", model_directory)
except subprocess.CalledProcessError as e:
    print("Error occurred during model training:", e)


Saving model to: ./trained_model with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 256 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 10 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }] }
Inputs: 59999 x 196
Targets: 59999 x 10
Training neural network with shape: NeuralNetworkShape { layers: [LayerShape { layer_type: Dense { input_size: 196, output_size: 256 }, activation: ActivationData { activation_type: Softmax, temperature: Some(2.5831520557403564) } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 256 }, activation: ActivationData { activation_type: Softmax, temperature: Some(2.5831520557403564) } }, LayerShape { layer_type: Dense { input_size: 256, output_size: 10 }, activation: ActivationData { activation_type: Sigmoid, temperature: None } }] }
Epoch: 0
Inputs: 59999 x 196
Targets:

KeyboardInterrupt: 

## Step 4: Evaluate the Model

In [ ]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
tolerance = 0.2

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/evaluate",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--tolerance", str(tolerance),
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model evaluation completed successfully")
except subprocess.CalledProcessError as e:
    print("Error occured during model evaluation:", e)


Accuracy: 0
Model training completed successfully and saved in: ./trained_model


## Step 5: Predict samples with the trained machine learning model

In [3]:
import subprocess

# Define parameters for `nn_generator`
model_directory = "./trained_model"
# create the trained model directory
import os
os.makedirs(model_directory, exist_ok=True)

input_file = "input.csv"
target_file = "target.csv"
predict_file = "predict.csv"
tolerance = 0.2

# Create the command for training the neural network
command = [
    "./ml_rust/target/release/predict",
    "--model-directory", model_directory,
    "--input-file", input_file,
    "--target-file", target_file,
    "--predict-file", predict_file,
    "--tolerance", str(tolerance),
]

# Run the command
try:
    #subprocess.run(command, check=True)
    # Create a copy of the current environment variables and add RUST_BACKTRACE=1
    env = os.environ.copy()
    env["RUST_BACKTRACE"] = "1"

    # Run the command
    subprocess.run(command, check=True, env=env)
    print("Model prediction finished")
except subprocess.CalledProcessError as e:
    print("Error occurred during model prediction:", e)


Accuracy: 0.9000000000009357
Model prediction finished


## Step 6: Benchmark model training with mnist dataset with keras

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import pandas as pd

# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0

# Convert targets to one-hot encoding
y_train_onehot = to_categorical(y_train, 10)

# put the data to csv files inputs.csv
input_df = pd.DataFrame(x_train.reshape(x_train.shape[0], -1))
input_df.to_csv("input.csv", index=False, header=False)
# and targets.csv
target_df = pd.DataFrame(y_train_onehot)
target_df.to_csv("target.csv", index=False, header=False)


# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),       # Input layer
    Dense(128, activation='relu'),      # Hidden layer 1
    Dense(64, activation='relu'),       # Hidden layer 2
    Dense(10, activation='softmax')     # Output layer
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


/usr/local/lib/python3.9/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.8588 - loss: 0.4761 - val_accuracy: 0.9536 - val_loss: 0.1493
Epoch 2/10
1220/1500 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9604 - loss: 0.1276

KeyboardInterrupt: 